# Test Neuroevolución

In [2]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler  

In [3]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import pickle

Importación del scaler
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
scaler = StandardScaler() 

#### Cargo los datos y retiro el Timestamp y el score

In [ ]:
df = pd.read_csv("gameStates.csv", sep=',', header=None)
df.columns = ['timeStamp','Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6','ray1','ray2','ray3','ray4','ray5','ray6','ray7','ray8','ray9','ray10','ray11','ray12','ray13','ray14','ray15','ray16','ray17','ray18','ray19','ray20','ray21','ray22','ray23','ray24','ray25','ray26','ray27','score',"VKey","HKey","Shooting"]
df.drop(['timeStamp','score'],axis=1,inplace=True)
df = df[1:5001]

In [ ]:
#Transforma el porblema multioputput en multilabel
def multioutput2multilabel(row):
    values = pd.Series([(row["VKey"] == "UpArrow"),(row["VKey"] == "DownArrow"),(row["HKey"] == "LeftArrow"),(row["HKey"] == "RightArrow"),row["Shooting"]])
    return pd.concat([row[:len(row)-3], values.astype(float)])
df = df.apply(multioutput2multilabel,axis=1)

In [ ]:
#Defino los elementos train_data y target_data
train = df.drop([0, 1, 2],axis=1).values
labels = df[[0,1,2]].values

In [ ]:
#Preprocesado obligatorio en este tipo de problemas -Leer más sobre ello-
scaler.fit(train)
train = scaler.transform(train)

In [ ]:
#Se crea el clasificador, se hace el entrenamiento y se guarda.
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)
trained_data = clf.fit(train, labels)
pickle.dump(trained_data, open("clf.sav", 'wb'))

In [ ]:
[coef.shape for coef in clf.coefs_]

In [ ]:
from copy import deepcopy
model = deepcopy(clf)

In [ ]:
"""
#Función de evaluación
from sklearn.metrics import accuracy_score
def MLPFitness(individual):
    coefs=gen2Coefs(individual,sizes,shapes)
    model.coefs_ = coefs
    preds = model.predict(Xprima)
    acc = accuracy_score(y, preds)
    return acc,
"""

In [ ]:
Xprima = train
y = labels
MLPFitness(coefs2gen(clf.coefs_,sizes,shapes))

## Definicion del algoritmo

In [4]:
#cargar los datos entrenados con el forest
clf = pickle.load(open("Forest.sav", 'rb'))
[coef.shape for coef in clf.coefs_]

shapes = [coef.shape for coef in clf.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in clf.coefs_]

#Transforma de en  a coefs y de coefs a gen
def gen2Coefs(gen,sizes,shapes):
    coefs = []
    splits = np.split(gen, [sizes[0]])
    for i in range(len(splits)):
        coefs.append(splits[i].reshape(shapes[i]))
    return coefs

def coefs2gen(coefs,sizes,shapes):
    return np.concatenate((coefs[0].flatten(),coefs[1].flatten()))

print(len(coefs2gen(clf.coefs_,sizes,shapes)))

pesosIniciales = coefs2gen(clf.coefs_,sizes,shapes).copy()

855


In [ ]:
#Defino los pesos del entrenamiento con el RandomForest.
pesosIniciales = coefs2gen(clf.coefs_,sizes,shapes).copy()

#Función de initPopulation
def initPopulation(pop, ind_random,n=10):
    pop = []
    for i in range(n):
        pop.append(ind_random())
    return pop

def getGuest():
    return coefs2gen(clf.coefs_,sizes,shapes)

def evalFunct(n):
        os.system("..\\test.exe")
        scores = pd.read_csv("myData.csv", sep=',', header=None)
        os.remove("myData.csv")
        return (np.mean(scores.values),)

        
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax) 
        
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 10)
toolbox.register("individual_rnd", tools.initRepeat, creator.Individual, toolbox.attr_float, n=sum(sizes))
toolbox.register("individual_guess", lambda :creator.Individual(pesosIniciales))
toolbox.register("population_mix",initPopulation,list,toolbox.individual_rnd)       


#print(toolbox.population_mix(n=10, n_guess=0))
genes = toolbox.population_mix(n=10)
toolbox.register("evaluate", evalFunct)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.2, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

random.seed(64)


pop = toolbox.population_mix(n=3)

hof = tools.HallOfFame(1, similar=np.array_equal)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=1, stats=stats, halloffame=hof)

In [ ]:
print(hof)
hof[0] 